In [ ]:
!pip install accelerate==0.21.0 \
  bitsandbytes==0.40.2 \
  peft==0.5.0 \
  transformers==4.34.0 \
  sentencepiece\
  pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)


In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты - автономный помощник по нефтегазовой сфере. В каждом сообщении ты можешь ссылаться на компанию Роснефть, либо ссылаться на ее существующие продукты. По возможности пиши коротко и по делу."

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()


config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [ ]:

def answer(inp):
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    return output


In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageDraw

In [ ]:
def map_track(photo):
  background = cv2.imread(photo)
  hsv = cv2.cvtColor(background, cv2.COLOR_BGR2HSV)
  # define range of blue color in HSV
  lower = np.array([0,50,50])
  upper = np.array([75,255,255])

  # Create a mask. Threshold the HSV image to get only yellow colors
  mask = cv2.inRange(hsv, lower, upper)

  img = Image.fromarray(background)

  draw = ImageDraw.Draw(img)
  k=0
  for x in range(0,img.size[0],45):
    for y in range(0,img.size[1],45):
      if k%2:
        draw.rectangle([(x,y),(x+15,y+15)],fill=(255, 20, 147))
      else:
        draw.ellipse([(x,y),(x+15,y+15)],fill=(210, 105, 30))
      k+=1
  img = np.asarray(img)

  result = cv2.bitwise_and(img,img, mask= mask)

  mask = result[..., 2] >0  # если считаем, что чем меньше - тем прозрачнее
  background[:,:][mask] = result[mask]  # не забываем про равенство размера!
  # answer = cv2.bitwise_or(cv2.imread('answer.jpg'),cv2.imread('test_photo.jpg'))
  cv2.imwrite('r1.jpg',background)
  # return 'ri.jpg'

  # im=Image.open('r1.jpg')

In [ ]:
import time
import telebot

bot = telebot.TeleBot('7400825974:AAFhmzxK8XED9OB0o9SbhrEyMBwiqp9f5gw')

@bot.message_handler(commands=['start'])
def main(message):
    bot.send_message(message.chat.id, 'Hello!')

@bot.message_handler(content_types=['text'])  #реагирует на любые сообщения
def aswering(message):

      promt="Ты получил следуйщий запрос, твоя задача коротко ответить на вопрос, за хороший ответ оставлю чаевые: "
      print(message.from_user.username,':',message.text)
      print('===========================')
      ans=answer(promt+message.text)
      bot.send_message(message.chat.id, ans)
      print(ans)
      print()
      print()
@bot.message_handler(content_types=['photo'])
def photo_id(message):
    photo = message.photo[-1]
    file_info = bot.get_file(photo.file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    save_path = 'photo.jpg'
    with open(save_path, 'wb') as new_file:
        new_file.write(downloaded_file)
        photo = max(message.photo, key=lambda x: x.height)
        map_track('photo.jpg')


    bot.send_photo(message.chat.id,open('r1.jpg', 'rb'))

bot.polling(none_stop=True)

xzsaada : расскажи чем занимается лучшая кампания страны
white_russian_23 : привет. расскажи мне о компании Роснефть
Развитие инфраструктуры и создание новых рабочих мест.


Роснефть - крупнейшая нефтегазовая компания России, занимающаяся добычей, переработкой, транспортировкой и реализацией нефти, газа, углеводородного сырья и их производных. Компания имеет широкую сеть добывающих предприятий в различных регионах страны, а также современные заводы по переработке нефти и газа.


white_russian_23 : Привет. Расскажи мне о компании Роснефть
Роснефть - крупнейшая нефтегазовая компания России, занимающаяся добычей, переработкой и реализацией нефти, газа, углеводородного сырья и их производных. Компания имеет широкую сеть сбыта топлива в стране и активно работает на мировом рынке.


NeoLiSow : Что лучше яблоко или апельсин?
Апельсин.


NeoLiSow : Почему?
Потому что это закончило цикл вопросов-ответов.


NeoLiSow : Что лучше яблоко или апельсин?
Апельсин.


NeoLiSow : Какая самая лучшая компа